# Result

This notebook shows the result of a parallel Bayesian Optimization over big data(>150M). The Parallel Bayesian Optimization strategy is called Multi-Points Expected Improvement using constant lier batch design. Method introduction see below.

**Information of Dataset**<br>
```
+-------------------------------------------------------------------------------+
| Name | Source | Size | TrainSet | TestSet | MissingValue | Balanced? |  Task  |
|------+--------+------+----------+---------+--------------+-----------+--------+
|albert| Openml |162.9M|  284910  | 140330  |     yes      |   yes     | binary |
+-------------------------------------------------------------------------------+

```
**Environment**<br>
To conduct this experiment, I run the code in Google Colab with 2 CPU.

**Toal Runtime:** 12h<br>

**Remark:**
To speed up the process I **only use LightGBM** with 7 hyperparameters.  <br>

**Experiment introduction** <br>
All strategies have run 20 times
- `njob_npoints10`: Using Parallel Optimization, batch size is 10
- `njon_npoints1`: Using Parallel Optimization, batch size is 1
- `with model_parallel`: only run two Surrogate Model(GP & RF) in parallel(CPU =2)
- `without_parallel`: No Parallel, but LightGBM algorithms run parallel (Remark: not the evaluation of LightGBM)



In [5]:
# Compare the result
import pandas as pd
'''
The table should be generated automatically. However, there is a max limit time (90min) of using Colab, 
so in this experiment I just set it by hand. Good news is, I just wirte a dump function. so that laterly
all the experiment results can be dumped locally. 
Due to the looooooong runtime, I don't dump the results this time. :)
'''

compare_result_ = pd.DataFrame(columns=['strategy','total_time(s)','best_score' ,'GP_time', 'GP_score','RF_time','RF_score'],
                              data =[['without_parallel',12752,0.7596,6042,0.7586,6710,0.7596],
                                     ['njob_npoints1',8887,0.7592,5311,0.7592,3575,0.756],
                                     ['njob_npoints10',8809,0.7592,5196,0.7592,3690,0.756],
                                     ['model_parallel',12256,0.7596,11383,0.7586,12256,0.7596]])
compare_result_.sort_values(by=['total_time(s)'])


,strategy,total_time(s),best_score,GP_time,GP_score,RF_time,RF_score
2,njob_npoints10,8809,0.7592,5196,0.7592,3690,0.7560
1,njob_npoints1,8887,0.7592,5311,0.7592,3575,0.7560
3,model_parallel,12256,0.7596,11383,0.7586,12256,0.7596
0,without_parallel,12752,0.7596,6042,0.7586,6710,0.7596


<font color=DarkSlateBlue size=5>Result analysis</font> <br>

From the result, we can see that the strategy: njob_npoints10  and strategy: njob_npoints1 outperform the other two strategies, which means Parallel Optimization by using Constant Lier has contributed to accelerating BO. <br>

In this case, the number of CPU is 2. If we just run the `Parallel Optimization Strategy`, it is ok. But if we want to parallel running `2 Surrogate model` and `Parallel Optimization Strategy`, 2 CPU are far from enough to use, that is,there should be at least 4 CPU available. We can image if there are enough CPU cores, the performance could be better.

More details please see Section 3-5


---

<font color=DarkSlateBlue size=5> Overall Introduction of Parallel Optimization by using Constant Lier</font>

- A Optimizer (BO) is created, which is called Optimizer A.
- Some initial samples(here 10) will be sampled randomly, and approximate the objective function f with a Surrogate model.
- A copy of optimizer A (called Optimizer B) is created, which is then asked for a point, and the point is told to the  Optimizer B with some fake objective (lie),which is used with lie objective value being minimum of observed objective values. The next point is asked from copy, it is also told to the copy with fake objective and so on, until [$\mathbf{X}_{i}$,$\mathbf{ylie}_{i}$] for $i = 1,2,...q$ are found.<br>
- Discard the copy optimizer B and evaluate all the q points by using objective function f, get the true [$\mathbf{X}_{i}$,$\mathbf{y}_{i}$] for $i = 1,2,...q$
- Find the next sampling point $\mathbf{X}_{j}$ by optimizing the acquisition function over the GP with all obeservated points (initial points + q points)
- Repeat ...
<br>

**Parallel Optimization**
```
Input: Function f, Current dataset (X,y)
Output: Global minimum x_min = argmin f
1  for Surrogate model in [GP, RF]
2  |  while (pending iterations):
3  |  |  Create a new copy of optimizer 'opt'
4  |  |  ......
5  |  |  Through Constant liar Batch design strategies, 
6  |  |  accumulate p points X_new=(x_1..x_p).
7  |  |  ......
8  |  |  Discard 'opt'
9  |  |  Evaluate f(X_new) in parallel get y_new
10 |  |  Append (X_new,y_new) to (X,y)
11 |  |  Fit updated model, discard opt
12 |  end while (**having k batches/k*p points evaluated now**)
13   end for
14 return global minimum x_min = argmin y
```
**Constant liar Batch Design**
```
Input: Copied optimizer 'opt'
Output: A batch of p points X_new=(x_1..x_p)
1  while (pending points to sample):
2  |  Choose next point x_i=argmin EI(X)
3  |  Find current best y_opt
4  |  Lie with constant y_i= min(y_opt)
5  |  Append (x_i,y_i) to (X,y) within 'opt'
6  |  Fit the copied model with (X,y) in 'opt'
7  end while
```

**More information see the paper:**
> https://hal.archives-ouvertes.fr/hal-00732512/document

---

# Default Setting for Google Colab

To conduct this experiment, I run the code in Google Colab with:


In [2]:
#default setting
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

!mkdir -p drive
!google-drive-ocamlfuse -o nonempty drive

import os
path = "drive/Masterarbeit/Automl_Framework/"
os.chdir(path)
#os.listdir(path)
!ls

!pip install scikit-optimize
!pip install scikit-learn==0.20.3


E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 130942 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.4-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.4-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.4-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

# Set Path and Read Data

In [41]:
# set path
import numpy as np
import base64
import os
from os.path import join
import sys

def mprint(msg):
    from datetime import datetime
    """info"""
    cur_time = datetime.now().strftime('%m-%d %H:%M:%S')
    print(f"INFO  [{cur_time}] {msg}")

def Config_DIRS():

    if len(sys.argv) == 1:
        # default local
        ROOT_DIR = os.getcwd()
        DIRS = {
            'input': join(ROOT_DIR, 'data'),
            'output': join(ROOT_DIR, 'save'),
            'program': join(ROOT_DIR, 'ingestion_program'),
            'submission': join(ROOT_DIR, 'automl')
        }
    elif len(sys.argv) == 3:
        # default local
        ROOT_DIR = os.getcwd()
        DIRS = {
            'input': join(ROOT_DIR, 'data'),
            'output': join(ROOT_DIR, 'save'),
            'program': join(ROOT_DIR, 'ingestion_program'),
            'submission': join(ROOT_DIR, 'automl')
        }

    elif len(sys.argv) == 5:
        # run in codalab
        DIRS = {
            'input': sys.argv[1],
            'output': sys.argv[2],
            'program': sys.argv[3],
            'submission': sys.argv[4]
        }
    elif len(sys.argv) == 6 and sys.argv[1] == 'local':
        # full call in local
        DIRS = {
            'input': sys.argv[2],
            'output': sys.argv[3],
            'program': sys.argv[4],
            'submission': sys.argv[5]
        }
    else:
        raise ValueError("Wrong number of arguments")
    sys.path.append(DIRS['submission'])
    print(DIRS)
    return(DIRS)

DIRS = Config_DIRS()

# read data
import reader
import imputation 
import encoder 


info = {
	"table_sep" : ',',
	"target_name" : 'class',
	"miss_values":'?'
}


datanames = [f for f in os.listdir(DIRS['input']) if not f.startswith('.')]
reader = reader.Reader(sep = info['table_sep'],
						miss_values=info['miss_values']
						)

for dataname in datanames:
        mprint(f'Read data: {dataname}')
        datapath = join(DIRS['input'], dataname)

data = reader.read_split([datapath],target_name=info['target_name'])
print(data.keys())




{'input': '/content/drive/Masterarbeit/Automl_Framework/data', 'output': '/content/drive/Masterarbeit/Automl_Framework/save', 'program': '/content/drive/Masterarbeit/Automl_Framework/ingestion_program', 'submission': '/content/drive/Masterarbeit/Automl_Framework/automl'}
INFO  [06-21 14:04:06] Read data: albert.csv

--------Start [read_split]:
------------Start [pre_clean]:

reading csv : albert.csv ...
cleaning data ...
------------End   [pre_clean]. Time elapsed: 72.53 sec.

> Number of common features : 78

gathering and crunching for train and test datasets ...
reindexing for train and test datasets ...
dropping training duplicates ...
dropping constant variables on training set ...

> % missing values on train set:
V67    0.8
V64    0.8
V12    0.8
V53    0.8
V69    0.8
dtype: float64

dropping columns with high missing rate >0.8...
> No need to dropping!

> Number of categorical features: 0
> Number of numerical features: 78
> Number of training samples : 284910
> Number of test s

# Setting Default Function for Optimizer

In [0]:
# import library and set the default function fot optimizer
import lightgbm as lgb
# Importing core libraries
import numpy as np
import pandas as pd
from time import time
import pprint
import joblib
import matplotlib.pyplot as plt

# Suppressing warnings because of skopt verbosity
import warnings
warnings.filterwarnings("ignore")

# Classifiers
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline


# Model selection
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

# Metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer

# Skopt functions
from skopt import BayesSearchCV
from skopt import gp_minimize # Bayesian optimization using Gaussian Processes
from skopt.space import Real, Categorical, Integer
from skopt.utils import use_named_args # decorator to convert a list of parameters to named arguments
from skopt.callbacks import DeadlineStopper # Stop the optimization before running out of a fixed budget of time.
from skopt.callbacks import VerboseCallback # Callback to control the verbosity
from skopt.callbacks import DeltaXStopper # Stop the optimization If the last two positions at which the objective has been evaluated are less than delta

random_state = 42
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
roc_auc = make_scorer(roc_auc_score, greater_is_better=True, needs_threshold=True)

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

import encoder
from util import timeit

enc =encoder.Categorical_encoder()
y=data['target']
X = enc.fit_transform(data['train'],y)


### default Function
  
def dump_result(data,inputname):
    import time
    from sklearn.externals.joblib import dump, load
    path = DIRS['output']
    datanames = sorted(os.listdir(DIRS['input']))[0].split('.')[0]
  #  timestr = time.strftime("%Y%m%d%H%M%S")
  #  filename = datanames + timestr + '.json'
    filename = datanames +'-'+inputname + '.json'
    dump(data, (path + '/' + filename))


def load_result(filename):
    dirs = init_dirs()
    path = join(dirs['output'])
    return load(path + '/' + filename)
  
# Callback function
def on_step(optim_result):
    score = opt.best_score_
    score_std = opt.cv_results_['std_test_score'][opt.best_index_]
    if score >= 0.99:
        print('Best Score >0.99,Interrupting!')
        return True

# Reporting util for different optimizers
def report_perf(optimizer, X, y, title, callbacks=None):
    """
    A wrapper for measuring time and performances of different optmizers
    
    optimizer = a sklearn or a skopt optimizer
    X = the training set 
    y = our target
    title = a string label for the experiment
    """

    start = time()
    if callbacks:
        mprint(f'start tuning {title}...')
        optimizer.fit(X, y, callback=callbacks)
    else:
    
        mprint(f'start tuning {title}...')
        optimizer.fit(X, y)

    time_cost = time() - start
    result = {}
    result['best_score'] = optimizer.best_score_
    result['best_score_std'] = optimizer.cv_results_['std_test_score'][optimizer.best_index_]
    result['best_params'] = optimizer.best_params_
    result['params'] = optimizer.cv_results_['params']
    result['time_cost(s)'] = round(time_cost, 1)
    result['all_cv_results'] = optimizer.cv_results_['mean_test_score'][:]
    result['mean_score_time'] = optimizer.cv_results_['mean_score_time'][:]
    result['mean_score_time'] = optimizer.cv_results_['mean_score_time'][:]

    print('>' + title + ':')
    time_cost = round(result['time_cost(s)'], 1)
    cand = len(result['all_cv_results'])
    best_cv = round(result['best_score'], 4)
    best_cv_sd = round(result['best_score_std'], 4)
    print(f'took{time_cost}s, candidates checked:{cand},best CV score: {best_cv} \u00B1 {best_cv_sd}')
    print("")
    return result

# Optimization


## without parellel

In [3]:
# Initialize pipeline
pipe = Pipeline([('model', lgb.LGBMClassifier(objective='binary', random_state=0))])

# Define search space for LGB:
search_space_LGB  = {"model": Categorical([lgb.LGBMClassifier(objective='binary', random_state=0)]),
                     "model__class_weight":Categorical(categories=['balanced', None]),
                    "model__learning_rate": Real(0.01, 1.0),
                    "model__boosting_type": Categorical(categories=['gbdt', 'dart']),
                    "model__n_estimators": Integer(10, 500),
                    "model__min_samples_split": Integer(2, 10),
                    "model__min_samples_leaf": Integer(1, 10),
                    "model__min_child_weight": Integer(0, 50)}

@timeit
def test_without_parallel():
    

    final_result = {}
    for baseEstimator in ['GP','RF']:    
        opt = BayesSearchCV(pipe,
                          search_spaces=[(search_space_LGB, 20)],
                          scoring=roc_auc,
                          cv=skf,
                          n_jobs=1,
                          return_train_score=False,
                          optimizer_kwargs={'base_estimator': baseEstimator,
                                           "acq_func" : "EI"},
                          random_state=4)

        result = report_perf(opt, X, y,'BayesSearchCV_'+baseEstimator,
                                                         callbacks=[DeltaXStopper(0.0001)]
                                                        )
        final_result[baseEstimator] = result
    return final_result
ttt1 = test_without_parallel()
save_result(ttt1,"test_without_parallel")


Start [test_without_parallel]:
INFO  [06-20 19:04:01] start tuning BayesSearchCV_GP...
>BayesSearchCV_GP:
took6042.2s, candidates checked:20,best CV score: 0.7586 ± 0.0019

INFO  [06-20 20:44:43] start tuning BayesSearchCV_RF...
>BayesSearchCV_RF:
took6710.7s, candidates checked:20,best CV score: 0.7596 ± 0.0016

End   [test_without_parallel]. Time elapsed: 12752.83 sec.


## with parallel n_jobs = -1,n_points =1

In [5]:
# Initialize pipeline 
pipe = Pipeline([('model', lgb.LGBMClassifier(objective='binary', random_state=0))])

# Define search space for LGB:
search_space_LGB  = {"model": Categorical([lgb.LGBMClassifier(objective='binary', random_state=0)]),
                     "model__class_weight":Categorical(categories=['balanced', None]),
                    "model__learning_rate": Real(0.01, 1.0),
                    "model__boosting_type": Categorical(categories=['gbdt', 'dart']),
                    "model__n_estimators": Integer(10, 500),
                    "model__min_samples_split": Integer(2, 10),
                    "model__min_samples_leaf": Integer(1, 10),
                    "model__min_child_weight": Integer(0, 50)}

@timeit
def test_withnjob():
    final_result = {}
    for baseEstimator in ['GP','RF']:    
        opt = BayesSearchCV(pipe,
                          search_spaces=[(search_space_LGB, 20)],
                          scoring=roc_auc,
                          cv=skf,
                          n_jobs=-1,
                          n_points = 10,
                          return_train_score=False,
                          optimizer_kwargs={'base_estimator': baseEstimator,
                                           "acq_func" : "EI"},
                          random_state=4)

        result = report_perf(opt, X, y,'BayesSearchCV_'+baseEstimator,
                                                         callbacks=[DeltaXStopper(0.0001)]
                                                        )
        final_result[baseEstimator] = result
    return final_result
test_withnjob = test_withnjob()


Start [test_withnjob]:
INFO  [06-20 22:36:34] start tuning BayesSearchCV_GP...
>BayesSearchCV_GP:
took5311.3s, candidates checked:20,best CV score: 0.7592 ± 0.0022

INFO  [06-21 00:05:05] start tuning BayesSearchCV_RF...
>BayesSearchCV_RF:
took3575.8s, candidates checked:20,best CV score: 0.756 ± 0.0015

End   [test_withnjob]. Time elapsed: 8887.07 sec.


## with njobs=-1,n_points=10

In [58]:
# Initialize pipeline 
pipe = Pipeline([('model', lgb.LGBMClassifier(objective='binary', random_state=0))])

# Define search space for LGB:
search_space_LGB  = {"model": Categorical([lgb.LGBMClassifier(objective='binary', random_state=0)]),
                     "model__class_weight":Categorical(categories=['balanced', None]),
                    "model__learning_rate": Real(0.01, 1.0),
                    "model__boosting_type": Categorical(categories=['gbdt', 'dart']),
                    "model__n_estimators": Integer(10, 500),
                    "model__min_samples_split": Integer(2, 10),
                    "model__min_samples_leaf": Integer(1, 10),
                    "model__min_child_weight": Integer(0, 50)}


@timeit
def test_njob_npoint():
    final_result = {}
    for baseEstimator in ['GP','RF']:    
        opt = BayesSearchCV(pipe,
                          search_spaces=[(search_space_LGB, 20)],
                          scoring=roc_auc,
                          cv=skf,
                          n_jobs=-1,
                          n_points = 10,
                          return_train_score=False,
                          optimizer_kwargs={'base_estimator': baseEstimator,
                                           "acq_func" : "EI"},
                          random_state=4)

        result = report_perf(opt, X, y,'BayesSearchCV_'+baseEstimator,
                                                         callbacks=[DeltaXStopper(0.0001)]
                                                        )
        final_result[baseEstimator] = result
    return final_result
njob_npoint = test_njob_npoint()
#save_result(njob_npoint,"test_njob_npoint")


--------Start [test_njob_npoint]:
INFO  [06-21 14:17:23] start tuning BayesSearchCV_GP...
>BayesSearchCV_GP:
took5196.0s, candidates checked:20,best CV score: 0.7592 ± 0.0022

INFO  [06-21 15:43:59] start tuning BayesSearchCV_RF...
>BayesSearchCV_RF:
took3690.4s, candidates checked:20,best CV score: 0.756 ± 0.0015

--------End   [test_njob_npoint]. Time elapsed: 8886.41 sec.


In [0]:
#dump the optimized results to .json 
dump_result(njob_npoint,'all_njob_npoint')


## with model_parallel

In [17]:
# model parallel
from joblib import Parallel, delayed
import multiprocessing

ee=['GP', 'RF']
def optm(baseEstimator):
    # Callback function
    def on_step(optim_result):
        score = opt.best_score_
        score_std = opt.cv_results_['std_test_score'][opt.best_index_]
        if score >= 0.99:
            print('Best Score >0.99,Interrupting!')
            return True


    # Reporting util for different optimizers
    def report_perf(optimizer, X, y, title, callbacks=None):
        """
        A wrapper for measuring time and performances of different optmizers

        optimizer = a sklearn or a skopt optimizer
        X = the training set 
        y = our target
        title = a string label for the experiment
        """


        start = time()
        if callbacks:
            print("")
            mprint(f'start tuning {title}...')
            optimizer.fit(X, y, callback=callbacks)
        else:
            print("")
            mprint(f'start tuning {title}...')
            optimizer.fit(X, y)

        time_cost = time() - start
        result = {}
        result['best_score'] = optimizer.best_score_
        result['best_score_std'] = optimizer.cv_results_['std_test_score'][optimizer.best_index_]
        result['best_params'] = optimizer.best_params_
        result['params'] = optimizer.cv_results_['params']
        result['time_cost(s)'] = round(time_cost, 1)
        result['all_cv_results'] = optimizer.cv_results_['mean_test_score'][:]
        result['mean_score_time'] = optimizer.cv_results_['mean_score_time'][:]
        result['mean_score_time'] = optimizer.cv_results_['mean_score_time'][:]

        print('>' + title + ':')
        time_cost = round(result['time_cost(s)'], 1)
        cand = len(result['best_params'])
        best_cv = round(result['best_score'], 4)
        best_cv_sd = round(result['best_score_std'], 4)
        print(f'took{time_cost}s, candidates checked:{cand},best CV score: {best_cv} \u00B1 {best_cv_sd}')
        print("")
        return result



    # Initialize a pipeline with a model
    pipe = Pipeline([('model', lgb.LGBMClassifier(objective='binary', random_state=0))])

    # Define search space for LGB:
    search_space_LGB  = {"model": Categorical([lgb.LGBMClassifier(objective='binary', random_state=0)]),
                       "model__class_weight":Categorical(categories=['balanced', None]),
                      "model__learning_rate": Real(0.01, 1.0),
                      "model__boosting_type": Categorical(categories=['gbdt', 'dart']),
                      "model__n_estimators": Integer(10, 500),
                      "model__min_samples_split": Integer(2, 10),
                      "model__min_samples_leaf": Integer(1, 10),
                      "model__min_child_weight": Integer(0, 50)}


    r = {}
  #  for baseEstimator in ['GP']:
    opt = BayesSearchCV(pipe,
                      search_spaces=[(search_space_LGB, 20)],
                      scoring=roc_auc,
                      cv=skf,
                      n_jobs=1,
  #                    n_points = 10,
                      return_train_score=False,
                      optimizer_kwargs={'base_estimator': baseEstimator,
                                         "acq_func" : "EI"},
                      random_state=4)

    result = report_perf(opt, X, y,'BayesSearchCV_'+baseEstimator,
                                                     callbacks=[DeltaXStopper(0.0001)]
                                             )
    r[baseEstimator] = result
    return r

num_cores = multiprocessing.cpu_count()

@timeit
def test_model_parallel ():
    start = time()
    results = Parallel(n_jobs=-1,verbose=0)(delayed(optm)(baseEstimator) for baseEstimator in ee)
    time_cost = time() - start
    return results,time_cost

model_parallel,time_cost = test_model_parallel()
print('Total time:',round(time_cost,1))
print('GP:' ,model_parallel[0]['GP']['best_score'])
print('RF:' ,model_parallel[1]['RF']['best_score'])

model_parallel_results={}
model_parallel_results['GP'] = model_parallel[0]['GP']
model_parallel_results['RF'] = model_parallel[1]['RF']
model_parallel_results['Total_time_cost'] = round(time_cost,1)
#model_parallel_results


--------Start [test_model_parallel]:
--------End   [test_model_parallel]. Time elapsed: 12257.63 sec.
Total time: 12257.6
GP: 0.7585502020558024
RF: 0.7596223157822072


In [0]:
#dump the optimized results to .json 
dump_result(model_parallel_results,'all_model_parallel_results')